In [108]:
import optuna 
import xgboost as xgb
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
import sklearn
from sklearn.metrics import mean_squared_error
import operator
from random import seed
from datetime import datetime
from random import random, choice

In [109]:
best_start = pd.read_csv('..\\kaggle_data\\0.69721.csv')
# seed random number generator
seed(1)

In [112]:
def iswinner(subject_df, plot_enalbed=False):
    best_val = 0.69751
    best = pd.read_csv(f'..\\kaggle_data\\{best_val}.csv')
    med_val = 0.70008
    med = pd.read_csv(f'..\\kaggle_data\\{med_val}.csv')
    medtwo_val = 0.72286
    medtwo = pd.read_csv(f'..\\kaggle_data\\{medtwo_val}.csv')
    medthree_val = 0.73846
    medthree = pd.read_csv(f'..\\kaggle_data\\{medthree_val}.csv')
    worst_val = 0.89798
    worst = pd.read_csv(f'..\\kaggle_data\\{worst_val}.csv')
    
    mse_sub_to_best = mean_squared_error(subject_df, best)
    rmse_sub_to_best = math.sqrt(mse_sub_to_best)

    mse_sub_to_med = mean_squared_error(subject_df, med)
    rmse_sub_to_med = math.sqrt(mse_sub_to_med)
    
    mse_sub_to_medtwo = mean_squared_error(subject_df, medtwo)
    rmse_sub_to_medtwo = math.sqrt(mse_sub_to_medtwo)
    
    mse_sub_to_medthree = mean_squared_error(subject_df, medthree)
    rmse_sub_to_medthree = math.sqrt(mse_sub_to_medthree)

    mse_sub_to_worst = mean_squared_error(subject_df, worst)
    rmse_sub_to_worst = math.sqrt(mse_sub_to_worst)
    
    distance_subbest_to_best = best_val - rmse_sub_to_best
    distance_submed_to_med = med_val - rmse_sub_to_med
    distance_submedtwo_to_medtwo = medtwo_val - rmse_sub_to_medtwo
    distance_submedthree_to_medthree = medthree_val - rmse_sub_to_medthree
    distance_subworst_to_worst = worst_val - rmse_sub_to_worst
    
    if plot_enalbed:
        fact_plot = [[best_val,best_val],[med_val,med_val],[medtwo_val,medtwo_val],[medthree_val,medthree_val],[worst_val,worst_val]]
        delta_plot = [[rmse_sub_to_best,rmse_sub_to_best],[rmse_sub_to_med,rmse_sub_to_med],[rmse_sub_to_medtwo,rmse_sub_to_medtwo],[rmse_sub_to_medthree,rmse_sub_to_medthree],[rmse_sub_to_worst,rmse_sub_to_worst]]
        plt.plot(fact_plot,linestyle='--', marker='o', color='b') 
        plt.plot(delta_plot,linestyle='--', marker='o', color='r')
        
        print(f"Distance, Sub_Best to Known_Best {distance_subbest_to_best}")
        print(f"Distance, Sub_Med to Known_Med {distance_submed_to_med}")
        print(f"Distance, Sub_Med to Known_Med {distance_submedtwo_to_medtwo}")
        print(f"Distance, Sub_Med to Known_Med {distance_submedthree_to_medthree}")
        print(f"Distance, Sub_Worst to Known_Worst {distance_subworst_to_worst}")
    
    if (distance_subbest_to_best > distance_submed_to_med) and (distance_submed_to_med > distance_submedtwo_to_medtwo) and (distance_submedtwo_to_medtwo > distance_submedthree_to_medthree) and (distance_submedthree_to_medthree > distance_subworst_to_worst):
        return True
    else:
        return False

In [ ]:
working_df = best_start.copy(deep=True)
for x in range(0, 10):
    for index, row in working_df.iterrows():
        do_work = random()
        do_work_cutoff = random()
        if do_work > do_work_cutoff:
            direction = random()
            if direction > 0.5:
                working_df.at[index,'target'] = row.target + random()/choice([1,2,4,8,16,32])
            else:
                working_df.at[index,'target'] = row.target - random()/choice([1,2,4,8,16,32])
    winner_val = iswinner(working_df, False)
    if winner_val:
        #iswinner(working_df, True)
        working_df = working_df.copy(deep=True)

    print(f"Winner? {winner_val} on Itter: {x}")
iswinner(working_df, True)
dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
print(working_df.head())
working_df.to_csv(f'..\\kaggle_data\\{dt_string}_submission.csv', index=False)

In [49]:
distance_mse_to_best = mean_squared_error(working_df, best)
distance_rmse_to_best = math.sqrt(distance_mse_to_best)
print(f'Distance to Best {distance_rmse_to_best}')
print(working_df.head())

Distance to Best 0.025523459706187426
   id    target
0   0  7.980049
1   2  7.857678
2   6  7.942353
3   7  8.312210
4  10  8.072800


In [50]:
from datetime import datetime
dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
print(working_df.head())
working_df.to_csv(f'..\\kaggle_data\\{dt_string}_submission.csv', index=False)

   id    target
0   0  7.980049
1   2  7.857678
2   6  7.942353
3   7  8.312210
4  10  8.072800
